In [34]:
import pandas as pd
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession
from datetime import datetime

In [23]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.environ["JAVA_HOME"] = "/opt/anaconda3/envs/bi_development/bin/java"  #--> '/opt/anaconda3/envs/bi_development/bin/java'
os.environ["SPARK_HOME"] = "/home/spark/spark-3.1.2-bin-hadoop2.7"
os.environ['PYSPARK_PYTHON']='python3.9'
os.environ['PYSPARK_DRIVER_PYTHON']='/opt/anaconda3/envs/bi_development/lib/python3.9'

In [24]:
path = '/mnt/lnxsasdtcn/sasdata/DISCO_M/ALZFenix/local/FtpPCSas/history_azfd_nov2019_sas'
spark = SparkSession.builder.getOrCreate()
sdf = spark.read.parquet(path)
kdf = sdf.to_koalas()

In [25]:
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInRead", "CORRECTED")

In [6]:
kdf.head()

,OE,CLAIM_CODE,FECHA_PROCESO,ACCIDENT_DATE,REPORTING_DATE,CURRENCY,GUARANTEE,PORTFOLIO_LOB,RESERVING_LOB,G_PAID,G_SS,G_CASE,G_CASE_RECOVERY,C_PAID,C_SS,C_CASE,FLAG,FIRST_CLOSE_DATE,LATEST_CLOSE_DATE,NATCAT,INSTRUMENTAL,CONSORCIABLE
0,AZFD,340004286,2006-12-01,1992-04-01,1992-04-01,EUR,30,1020,1020_BI,0.0,0.0,6000.0,0.0,0.0,0.0,0.0,1,,,0,0,0
1,AZFD,340005825,2001-03-01,1994-02-01,1994-02-01,EUR,2,1020,1020_MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,20010201,20010201,0,0,0
2,AZFD,340005825,2001-06-01,1994-02-01,1994-02-01,EUR,2,1020,1020_MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,20010201,20010201,0,0,0
3,AZFD,340005825,2001-09-01,1994-02-01,1994-02-01,EUR,2,1020,1020_MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,20010201,20010201,0,0,0
4,AZFD,340005825,2001-12-01,1994-02-01,1994-02-01,EUR,2,1020,1020_MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,20010201,20010201,0,0,0


In [26]:
len(kdf)
# con pandas habia 12569358 filas y con koalas:

12569358

Converting Fenix parquet files into csv format by month

In [29]:
%time kdf = kdf.assign(MES_PROCESO= kdf["FECHA_PROCESO"].dt.strftime('%Y%m'))

# /home/e106064/.local/lib/python3.9/site-packages/pyspark/sql/pandas/functions.py:389: 
# UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF
# instead of specifying pandas UDF type which will be deprecated in the future releases. 
# See SPARK-28264 for more details.

CPU times: user 15.8 ms, sys: 7.59 ms, total: 23.4 ms
Wall time: 66.9 ms


/home/e106064/.local/lib/python3.9/site-packages/pyspark/sql/pandas/functions.py:389: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [47]:
kdf.head()

,MES_PROCESO,OE,CLAIM_CODE,FECHA_PROCESO,ACCIDENT_DATE,REPORTING_DATE,CURRENCY,GUARANTEE,PORTFOLIO_LOB,RESERVING_LOB,G_PAID,G_SS,G_CASE,G_CASE_RECOVERY,C_PAID,C_SS,C_CASE,FLAG,FIRST_CLOSE_DATE,LATEST_CLOSE_DATE,NATCAT,INSTRUMENTAL,CONSORCIABLE
0,200612,AZFD,340004286,2006-12-01,1992-04-01,1992-04-01,EUR,30,1020,1020_BI,0.0,0.0,6000.0,0.0,0.0,0.0,0.0,1,,,0,0,0
1,200103,AZFD,340005825,2001-03-01,1994-02-01,1994-02-01,EUR,2,1020,1020_MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,20010201,20010201,0,0,0
2,200106,AZFD,340005825,2001-06-01,1994-02-01,1994-02-01,EUR,2,1020,1020_MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,20010201,20010201,0,0,0
3,200109,AZFD,340005825,2001-09-01,1994-02-01,1994-02-01,EUR,2,1020,1020_MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,20010201,20010201,0,0,0
4,200112,AZFD,340005825,2001-12-01,1994-02-01,1994-02-01,EUR,2,1020,1020_MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,20010201,20010201,0,0,0


In [142]:
%time months = kdf.MES_PROCESO.unique().to_numpy()

CPU times: user 26.2 ms, sys: 11.8 ms, total: 38 ms
Wall time: 1min 21s


In [163]:
dt_ini = datetime.now()

for month in months:
    kdf[kdf.MES_PROCESO == month].to_csv(r'data/fenix_file_{}.csv'.format(month), header = True, sep = ';')    

dt_end = datetime.now()    
print(dt_end - dt_ini)

3:30:21.510357


In [164]:
spark.stop()